In [57]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit,sum,count,min,max,avg,udf
from pyspark.sql.types import IntegerType
spark=SparkSession.builder.appName('miniproject').getOrCreate()
df=spark.read.options(header=True, inferSchema=True).csv('Files/OfficeDataProject.csv')
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [16]:
ct=df.count()

print(f'Total number of employees {ct}')

Total number of employees 1000


In [21]:
dp=df.select('department').distinct().count()
print(f'Total number of departments {dp}')

Total number of departments 6


In [22]:
dp=df.select('department').distinct().show()


+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [25]:
dp=df.groupBy('department').agg(count('*').alias('total employees')).show()

+----------+---------------+
|department|total employees|
+----------+---------------+
|     Sales|            169|
|        HR|            171|
|   Finance|            162|
|Purchasing|            166|
| Marketing|            170|
|  Accounts|            162|
+----------+---------------+



In [26]:
dp=df.groupBy('state').agg(count('*').alias('total employees')).show()

+-----+---------------+
|state|total employees|
+-----+---------------+
|   LA|            205|
|   CA|            205|
|   WA|            208|
|   NY|            173|
|   AK|            209|
+-----+---------------+



In [28]:
dp=df.groupBy('state','department').agg(count('*').alias('total employees')).show()

+-----+----------+---------------+
|state|department|total employees|
+-----+----------+---------------+
|   CA|     Sales|             42|
|   CA| Marketing|             33|
|   NY|  Accounts|             34|
|   NY|     Sales|             27|
|   CA|   Finance|             35|
|   CA|  Accounts|             35|
|   CA|Purchasing|             32|
|   WA|        HR|             47|
|   AK|Purchasing|             30|
|   WA|  Accounts|             27|
|   WA|Purchasing|             38|
|   AK|     Sales|             38|
|   AK|  Accounts|             37|
|   WA| Marketing|             39|
|   LA|        HR|             41|
|   LA|     Sales|             35|
|   AK|        HR|             25|
|   LA|   Finance|             29|
|   AK|   Finance|             37|
|   LA|Purchasing|             45|
+-----+----------+---------------+
only showing top 20 rows



In [38]:
df.orderBy(df.salary.asc()).groupBy('department').agg(min('salary'), max('salary')).show()

+----------+-----------+-----------+
|department|min(salary)|max(salary)|
+----------+-----------+-----------+
|     Sales|       1103|       9982|
|        HR|       1013|       9982|
|   Finance|       1006|       9899|
|Purchasing|       1105|       9985|
| Marketing|       1031|       9974|
|  Accounts|       1007|       9890|
+----------+-----------+-----------+



In [56]:
bn=df.filter(df.state=='NY').groupBy('state').agg(avg('bonus').alias('avg'))
av=bn.select('avg').collect()[0]['avg']
df.filter((df.state=='NY') & (df.department=='Finance') & 
          (df.bonus> av)).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1082|          Nena Rocha|   Finance|   NY|  3417| 25| 1647|
|       1087|       Leif Lemaster|   Finance|   NY|  8642| 45| 1782|
|       1100|Ellingsworth Meli...|   Finance|   NY|  7845| 32| 1358|
|       1127|        Escoto Gilma|   Finance|   NY|  3426| 41| 1285|
|       1161|     Georgeanna Laub|   Finance|   NY|  2469| 26| 1679|
|       1175|     Durio Tenenbaum|   Finance|   NY|  2253| 42| 1684|
|       1180|       Juliana Grigg|   Finance|   NY|  8178| 42| 1617|
|       1215|        Tiffani Benz|   Finance|   NY|  1665| 41| 1969|
|       1220|          Nitz Ilana|   Finance|   NY|  2443| 50| 1342|
|       1342|   Phylicia Antonina|

In [59]:
def raisesal(salary,age):
    if age>45:
        return salary+500
    else:
        return salary
    
salRaise=udf(lambda x,y: raisesal(x,y),IntegerType())
df.withColumn('new sal',salRaise(df.salary,df.age)).show()

+-----------+-------------------+----------+-----+------+---+-----+-------+
|employee_id|      employee_name|department|state|salary|age|bonus|new sal|
+-----------+-------------------+----------+-----+------+---+-----+-------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|   6131|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|   4027|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|   3122|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|   6217|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|   5685|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|   2843|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|   7201|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|   3444|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|   4204|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|   5061|
|       1010

In [65]:
df.filter(df.age>45).write.option('header',True).mode('append').csv('Files/Employeewithage45')

In [66]:
spark.read.option('header',True).csv('Files/Employeewithage45/*').show()

+-----------+------------------+----------+-----+------+---+-----+
|employee_id|     employee_name|department|state|salary|age|bonus|
+-----------+------------------+----------+-----+------+---+-----+
|       1003|       Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1008| Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1011|  Barringer Escoto|Purchasing|   WA|  1685| 49| 1706|
|       1018|Vankirk Jacquelyne|Purchasing|   NY|  8636| 47| 1192|
|       1025|   Dionne Lemaster|     Sales|   AK|  5134| 48| 1356|
|       1030|        Trena Benz|  Accounts|   NY|  4376| 49| 1624|
|       1039|      Dynes Katlyn|  Accounts|   AK|  3039| 48|  834|
|       1058|      Clune Norene|   Finance|   AK|  1605| 49|  801|
|       1074|      Rocha Dionne|  Accounts|   CA|  3470| 49|  706|
|       1088|       Imai Locust|     Sales|   NY|  9982| 49| 1809|
|       1090| Clemencia Rudolph|   Finance|   NY|  1296| 50| 1209|
|       1099|    Zollner Marvis|Purchasing|   CA|  4230| 50|  